## Auhtor
Muhamad Nur Syami

Copyright 2025

ulasan penulis Muhamad Nur Syami :
Kode ini dipergunakan untuk menyelesaikan jenjang studi S1 Sarjana Teknik/Computer Science Program Studi Teknik Informatika Universitas Maritim Raja Ali Haji

In [ ]:
# =======================================
# ✅ INSTALL DEPENDENCIES & MOUNT DRIVE
# =======================================
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install pycocotools roboflow

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s e

In [ ]:
from google.colab import drive
import os, shutil

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# =======================================
# ✅ DOWNLOAD DATASET FROM ROBOFLOW
# =======================================
# Ini hanya demo saja, dan dataset ini sudah diganti dan tidak bisa digunakand dikarenakan hak cipta dataset,
# jadi silahkan pakai dataset anda yang telah anda labeling di aplikasi robotflow
# dan gunakan format coco json dengan api key dari robotflow.
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="asldjwuaksdj913")
project = rf.workspace("jhon").project("anonymous-dummy")
version = project.version(1)
dataset = version.download("coco")






loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Mangrove-Bintan-1 in coco:: 100%|██████████| 5408/5408 [00:01<00:00, 4618.75it/s]


In [ ]:
# =======================================
# ✅ IMPORT LIBRARY UTAMA
# =======================================

import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models import ResNet50_Weights
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from PIL import Image
import matplotlib.pyplot as plt
import json
import numpy as np
import os

In [ ]:
# =======================================
# ✅ FUNGSI VALIDASI STRUKTUR DAN STATISTIK DATASET (DITAMBAH DISTRIBUSI KATEGORI)
# =======================================
def validate_and_summarize_coco(json_path):
    with open(json_path) as f:
        data = json.load(f)

    print(f"\n📁 Validasi File: {json_path}")
    for key in ['images', 'annotations', 'categories']:
        if key not in data:
            print(f"❌ Missing key: {key}")
            return

    print("✅ Struktur Valid")
    print(f"Total Images: {len(data['images'])}")
    print(f"Total Annotations: {len(data['annotations'])}")
    print(f"Total Categories: {len(data['categories'])}")

    # Tampilkan daftar kategori
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    print(f"Categories: {list(categories.values())}")

    # Hitung distribusi anotasi per kategori
    annotation_counts = {cat_id: 0 for cat_id in categories.keys()}
    for ann in data['annotations']:
        cat_id = ann['category_id']
        if cat_id in annotation_counts:
            annotation_counts[cat_id] += 1

    print("\n📊 Distribusi Anotasi per Kategori:")
    for cat_id, count in annotation_counts.items():
        print(f"  {categories[cat_id]}: {count} anotasi")

# Silahkan sesuaikan path dataset anda
validate_and_summarize_coco("/content/Dummy-Dummy-1/train/_annotations.coco.json")
validate_and_summarize_coco("/content/Dummy-Dummy-1/valid/_annotations.coco.json")
validate_and_summarize_coco("/content/Dummy-Dummy-1/test/_annotations.coco.json")



📁 Validasi File: /content/Mangrove-Bintan-1/train/_annotations.coco.json
✅ Struktur Valid
Total Images: 3780
Total Annotations: 4014
Total Categories: 10
Categories: ['Avicennia-lanata', 'Avicennia-lanata', 'Bruguiera-cylindrica', 'Bruguiera-gymnorrhiza', 'Lumnitzera-liitorea', 'Rhizophora-apiculata', 'Rhizophora-mucronata', 'Scyphiphora-hydrophyllacea', 'Sonneratia-alba', 'Xylocarpus-granatum']

📊 Distribusi Anotasi per Kategori:
  Avicennia-lanata: 0 anotasi
  Avicennia-lanata: 456 anotasi
  Bruguiera-cylindrica: 462 anotasi
  Bruguiera-gymnorrhiza: 435 anotasi
  Lumnitzera-liitorea: 442 anotasi
  Rhizophora-apiculata: 413 anotasi
  Rhizophora-mucronata: 464 anotasi
  Scyphiphora-hydrophyllacea: 438 anotasi
  Sonneratia-alba: 414 anotasi
  Xylocarpus-granatum: 490 anotasi

📁 Validasi File: /content/Mangrove-Bintan-1/valid/_annotations.coco.json
✅ Struktur Valid
Total Images: 1080
Total Annotations: 1163
Total Categories: 10
Categories: ['Avicennia-lanata', 'Avicennia-lanata', 'Brugu

In [ ]:
# =======================================
# ✅ TRANSFORMASI & DATASET
# =======================================
class CocoTransform:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(root=img_dir, annFile=ann_file, transforms=CocoTransform())

def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = get_coco_dataset("/content/Dummy-Dummy-1/train", "/content/Dummy-Dummy-1/train/_annotations.coco.json")
val_dataset = get_coco_dataset("/content/Dummy-Dummy-1/valid", "/content/Dummy-Dummy-1/valid/_annotations.coco.json")

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
# =======================================
# ✅ INISIALISASI MODEL
# =======================================


def get_model(num_classes):

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='FasterRCNN_ResNet50_FPN_Weights.COCO_V1')

    # Ganti detection head (box predictor)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

num_classes = 10
model = get_model(num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
model.to(device)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 87.7MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
# =======================================
# ✅ TRAINING DAN EVALUASI PER EPOCH
# =======================================
all_losses = []
map_per_epoch = []
ap_per_class_epoch = []

# Background default, dan tidak perlu di hapus dan di ganti, silakan lanjut dan dimulai dari nomor 1 untuk menganti kelas yang anda inginkan.
COCO_CLASSES = {
    0: "Background",
    1: "Dummy-Dummy",
    2: "Dummy-Dummy",
    3: "Dummy-Dummy",
    4: "Dummy-Dummy",
    5: "Dummy-Dummy",
    6: "Dummy-Dummy",
    7: "Dummy-Dummy",
    8: "Dummy-Dummy",
    9: "Dummy-Dummy",
}

num_epochs = 25
# Mmebuat Folder untuk menyimpan hasil training seperti model dan visualiasi berdasarkan best model.:
save_dir = "/content/drive/MyDrive/training_model/tanpa_augmentasi_25epoch_4bacthsize"
os.makedirs(save_dir, exist_ok=True)

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0.0
    for images, targets in data_loader:
        images = [img.to(device) for img in images]
        processed_targets = []
        valid_images = []
        for i, target in enumerate(targets):
            boxes, labels = [], []
            for obj in target:
                x, y, w, h = obj["bbox"]
                if w > 0 and h > 0:
                    boxes.append([x, y, x+w, y+h])
                    labels.append(obj["category_id"])
            if boxes:
                processed_targets.append({
                    "boxes": torch.tensor(boxes, dtype=torch.float32).to(device),
                    "labels": torch.tensor(labels, dtype=torch.int64).to(device)
                })
                valid_images.append(images[i])
        if not processed_targets:
            continue
        loss_dict = model(valid_images, processed_targets)
        losses = sum(loss for loss in loss_dict.values())
        total_loss += losses.item()
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    avg_loss = total_loss / len(data_loader)
    all_losses.append(avg_loss)
    print(f"Epoch [{epoch+1}] Training Loss: {avg_loss:.4f}")

def evaluate_model(model, data_loader, ann_file):
    model.eval()
    coco_gt = COCO(ann_file)
    results = []
    for images, targets in data_loader:
        images = [img.to(device) for img in images]
        with torch.no_grad():
            outputs = model(images)
        for i, output in enumerate(outputs):
            image_id = targets[i][0]["image_id"]
            for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
                x_min, y_min, x_max, y_max = box.cpu().numpy()
                width = x_max - x_min
                height = y_max - y_min
                results.append({
                    "image_id": int(image_id),
                    "category_id": int(label),
                    "bbox": [x_min, y_min, width, height],
                    "score": float(score)
                })
    coco_dt = coco_gt.loadRes(results)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    #Kode MAP
    global_map = coco_eval.stats[0] * 100 # mAP pada IoU=0.5:0.95
    map_per_epoch.append(global_map)
    # Kode AP
    ap_per_class = {}
    for i, cat_id in enumerate(coco_eval.params.catIds):
        ap = np.mean(coco_eval.eval['precision'][0, :, i, 0, -1]) * 100
        class_name = COCO_CLASSES.get(cat_id, f"Class {cat_id}")
        ap_per_class[class_name] = ap
        print(f"  AP @{class_name}: {ap:.2f}%")

    ap_per_class_epoch.append(ap_per_class)

In [ ]:
best_map = 0.0
best_model_path = os.path.join(save_dir, "best_model.pth")
save_per_epoch = False  # Set ke False jika tidak ingin simpan tiap epoch

# Loop training
for epoch in range(num_epochs):
    print(f"\n==== Epoch {epoch+1}/{num_epochs} ====")
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()

    evaluate_model(model, val_loader, "/content/Mangrove-Bintan-1/valid/_annotations.coco.json")

    current_map = map_per_epoch[-1]
    print(f"Epoch {epoch+1}: mAP = {current_map:.2f}%")

    # Simpan model terbaik berdasarkan mAP
    if current_map > best_map:
        best_map = current_map
        torch.save(model.state_dict(), best_model_path)
        print(f"🏆 Best model saved at epoch {epoch+1} with mAP: {best_map:.2f}%")

    # Simpan model setiap epoch jika diperlukan
    if save_per_epoch:
        save_path = os.path.join(save_dir, f"fasterrcnn_resnet50_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved: {save_path}")



==== Epoch 1/25 ====
Epoch [1] Training Loss: 0.1883
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.17s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.14s).
Accumulating evaluation results...
DONE (t=0.28s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.410
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.760
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.388
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.409
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.576
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.652
 Average R

In [ ]:
import matplotlib.pyplot as plt

# =======================================
# ✅ VISUALISASI LOSS, mAP, AP PER KELAS
# =======================================
save_dir = "/content/drive/MyDrive/training_model/tanpa_augmentasi_25epoch_4bacthsize"
# Visualisasi Training Loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), all_losses, marker='o', color='b')
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
training_loss_path = "/content/drive/MyDrive/training_model/tanpa_augmentasi_25epoch_4bacthsize/training_loss_plot.png"
plt.savefig(training_loss_path)
plt.show()
plt.close()
print(f"Training Loss plot saved at: {training_loss_path}")

# Visualisasi Validation mAP [IoU=0.50:0.95]
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), map_per_epoch, marker='o', color='green')
plt.title("Validation mAP [IoU=0.50:0.95]")
plt.xlabel("Epoch")
plt.ylabel("mAP (%)")
plt.grid(True)
validation_map_path = "/content/drive/MyDrive/training_model/tanpa_augmentasi_25epoch_4bacthsize/validation_map_plot.png"
plt.savefig(validation_map_path)
plt.show()
plt.close()
print(f"Validation mAP plot saved at: {validation_map_path}")

# Visualisasi AP per Class
plt.figure(figsize=(12, 8))
for class_name in ap_per_class_epoch[0].keys():
    ap_values = [epoch_ap.get(class_name, 0) for epoch_ap in ap_per_class_epoch]
    plt.plot(range(1, num_epochs + 1), ap_values, label=class_name)

plt.title("Validation AP per Class")
plt.xlabel("Epoch")
plt.ylabel("AP (%)")


plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
plt.grid(True)

# Menyimpan dan menampilkan plot
ap_per_class_path = "/content/drive/MyDrive/training_model/tanpa_augmentasi_25epoch_4bacthsize/ap_per_class_plot.png"
plt.tight_layout()  # Menghindari pemotongan
plt.savefig(ap_per_class_path, bbox_inches='tight')
plt.show()
plt.close()

print(f"AP per Class plot saved at: {ap_per_class_path}")


In [ ]:
# Predict gambar  menggunakan gambar testing atau yang anda upload untuk menguji model

img_path = "/content/hhsmfuk4nfd.jpg"
image_tensor, image_raw = prepare_image(img_path)
with torch.no_grad():
    prediction = model(image_tensor)
draw_boxes(image_raw, prediction)